In [ ]:
https://universe.roboflow.com/test1-2vmvi/test1-9vnrj/browse?queryText=&pageSize=50&startingIndex=0&browseQuery=true

In [ ]:
https://universe.roboflow.com/insulator-defect-detection-cyjg2/smear-detection/dataset/2

In [1]:
# !pip install -U ultralytics

In [1]:
from ultralytics import YOLO
import os
import cv2
import shutil
from sklearn.model_selection import train_test_split
import numpy as np
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [20]:
# IMAGES_DIR = 'cv_open_dataset/open_img/'  # Путь к вашему датасету с изображениями
# MASKS_DIR = 'cv_open_dataset/open_msk/'  # Путь к вашему датасету с масками
# IMAGES_DIR = 'cv_synt_dataset/synt_img/'  # Путь к вашему датасету с изображениями
# MASKS_DIR = 'cv_synt_dataset/synt_msk/'  # Путь к вашему датасету с масками
IMAGES_DIR = 'cv_new_dataset_test/rgbImages/'  # Путь к вашему датасету с изображениями
MASKS_DIR = 'cv_new_dataset_test/rgbLabels/'  # Путь к вашему датасету с масками
OUTPUT_DIR = './datasets/train_data'  # Путь к выходной директории
TRAIN_SIZE = 0.8  # Процент обучающей выборки

In [21]:
os.makedirs(os.path.join(OUTPUT_DIR, 'images/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'images/val'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/train'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'labels/val'), exist_ok=True)

In [22]:
image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith(('.jpg', '.png'))]
mask_files = [f for f in os.listdir(MASKS_DIR) if f.endswith('.png')]
if len(image_files) != len(mask_files):
    print(len(image_files))
    print(len(mask_files))
    print("Количество изображений и масок не совпадает.")
    for mask in mask_files:
        if mask.replace(".png", ".jpg") not in image_files:
            print(mask)

In [23]:
train_images, val_images = train_test_split(image_files, train_size=TRAIN_SIZE, random_state=42)

In [2]:
def convert_mask_to_yolo(mask_path, out):
    # Открываем изображение
    image = cv2.imread(mask_path)

    if image is None:
        print(f"Не удалось открыть изображение: {mask_path}")
        return
    
    height, width = image.shape[:2]

    # Создаем маску для черного цвета
    black_mask = cv2.inRange(image, (0, 0, 0), (50, 50, 50))

    # Создаем новое изображение, где черный цвет остается, а остальные цвета становятся белыми
    new_image = np.ones_like(image) * 255  # Начинаем с белого изображения
    new_image[black_mask > 0] = [0, 0, 0]  # Заменяем черные пиксели

    # Преобразуем в градации серого для нахождения контуров
    gray_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

    # Находим контуры
    contours, _ = cv2.findContours(gray_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Сохраняем контуры в текстовом формате
    output_file_name = os.path.splitext(os.path.basename(mask_path))[0] + '.txt'
    output_file_path = os.path.join(OUTPUT_DIR, out, output_file_name)

    with open(output_file_path, 'w') as f:
        for index, contour in enumerate(contours):
            # Получаем координаты всех точек контура
            contour_points = contour.reshape(-1, 2)
            # Нормализуем координаты
            normalized_points = [(x / width, y / height) for x, y in contour_points]
            points_str = ' '.join(f"{x:.3f} {y:.3f}" for x, y in normalized_points)
            f.write(f"0 {points_str}\n")

In [25]:
# Копирование изображений и масок в соответствующие папки
for img in train_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/train', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/train')

In [26]:
for img in val_images:
    shutil.copy(os.path.join(IMAGES_DIR, img), os.path.join(OUTPUT_DIR, 'images/val', img))
    mask_name = img.replace('.jpg', '.png')
    convert_mask_to_yolo(os.path.join(MASKS_DIR, mask_name), 'labels/val')


In [10]:
data_yaml_content = f"""
train: train_data/images/train
val: train_data/images/val

nc: 1  # Обновите количество классов (1 для загрязнения)
names: ['contaminated']  # Обновите названия классов
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml_content)

print("Датасет успешно разбит и сохранен в структуре проекта.")

Датасет успешно разбит и сохранен в структуре проекта.


In [3]:
model = YOLO("baseline6.pt")


In [4]:
# torch.cuda.is_available()

In [6]:
train_results = model.train(
    data="./data.yaml",  # path to dataset YAML
    epochs=100,  # number of training epochs
    batch=8,
    lr0=3e-4,
    patience=4,
    imgsz=640,  # training image size
    device=0,  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
    # evolve=True,
)

New https://pypi.org/project/ultralytics/8.3.47 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.44 🚀 Python-3.8.19 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5929MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=segment, mode=train, model=baseline6.pt, data=./data.yaml, epochs=100, time=None, patience=4, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train5, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, cla

train: Scanning /home/dm/Документы/some/nornickel_2024/train_dataset/baseline/da

albumentations: module 'numpy' has no attribute 'bool'.
`np.bool` was a deprecated alias for the builtin `bool`. To avoid this error in existing code, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



/home/dm/.local/lib/python3.8/site-packages/skimage/morphology/_skeletonize.py:241: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  0, 1, 1, 0, 0, 1, 0, 0, 0], dtype=np.bool)
val: Scanning /home/dm/Документы/some/nornickel_2024/train_dataset/baseline/data


Plotting labels to runs/segment/train5/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.0003' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 90 weight(decay=0.0), 101 weight(decay=0.0005), 100 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs/segment/train5
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


...

      5/100      2.95G     0.7418      1.491      0.542      1.097          6   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all        565       1312      0.951      0.877      0.924      0.721      0.951      0.877       0.92      0.674
EarlyStopping: Training stopped early as no improvement observed in last 4 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=4) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



5 epochs completed in 0.088 hours.
Optimizer stripped from runs/segment/train5/weights/last.pt, 6.0MB
Optimizer stripped from runs/segment/train5/weights/best.pt, 6.0MB

Validating runs/segment/train5/weights/best.pt...
Ultralytics 8.3.44 🚀 Python-3.8.19 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5929MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        565       1312      0.949      0.885      0.927       0.74      0.958      0.872      0.916      0.658
Speed: 0.3ms preprocess, 3.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/segment/train5


In [7]:
results = model.val(data="./data.yaml")
# print(results)

Ultralytics 8.3.44 🚀 Python-3.8.19 torch-1.13.1+cu117 CUDA:0 (NVIDIA GeForce GTX 1660 SUPER, 5929MiB)
YOLO11n-seg summary (fused): 265 layers, 2,834,763 parameters, 0 gradients, 10.2 GFLOPs


val: Scanning /home/dm/Документы/some/nornickel_2024/train_dataset/baseline/data
                 Class     Images  Instances      Box(P          R      mAP50  m


                   all        565       1312      0.949      0.883      0.927       0.74      0.957      0.872      0.916      0.658
Speed: 0.2ms preprocess, 4.2ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to runs/segment/train52


In [34]:
def infer_image(image_path):
    # Загрузка изображения
    image = cv2.imread(image_path)

    # Инференс
    return model(image)



In [16]:
# Функция для создания маски с черным фоном
def create_mask(image_path, results):
    # Загружаем изображение и переводим в градации серого
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    # Создаем пустую маску с черным фоном
    mask = np.zeros((height, width), dtype=np.uint8)

    # Проходим по результатам и создаем маску
    for result in results:
        masks = result.masks  # Получаем маски из результатов
        if masks is not None:
            for mask_array in masks.data:  # Получаем маски как массивы
                mask_i = mask_array.detach().cpu().numpy()  # Преобразуем маску в numpy массив
                
                # Изменяем размер маски под размер оригинального изображения
                mask_i_resized = cv2.resize(mask_i, (width, height), interpolation=cv2.INTER_LINEAR)
                
                # Накладываем маску на пустую маску (255 для белого)
                mask[mask_i_resized > 0] = 255

    return mask

In [17]:
results = infer_image("./cv_test_dataset/test_img/_07.jpg")
mask_image = create_mask("./cv_test_dataset/test_img/_07.jpg", results)

# Сохраняем маску в формате PNG
mask_output_path = './mask_image.png'  # Укажите путь для сохранения маски
cv2.imwrite(mask_output_path, mask_image)


0: 480x640 5 contaminateds, 23.3ms
Speed: 3.9ms preprocess, 23.3ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)


True

In [8]:
model.save("baseline6.pt")

In [39]:
model1 = YOLO("baseline1.pt")